## 准备数据

In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=(
        "https://lilianweng.github.io/posts/2023-06-23-agent/",
        "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    ),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

docs = text_splitter.split_documents(documents)

docs[1]

USER_AGENT environment variable not set, consider setting it to identify your requests.


Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Component One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a 

## 实例化Milvus的向量数据库，该数据库会将文档加载到Milvs向量数据库并在后台构建索引

In [3]:
import sys
from langchain_milvus import Milvus
sys.path.append("..")
from llm_utils import qwen_embeddings,llm

docs = docs[:10]  # 限制文档数量，避免批量过大报错
vectorstore = Milvus.from_documents(
    documents=docs,
    embedding=qwen_embeddings,
    connection_args={
        "uri": "http://localhost:19530",  # http://localhost:19530
    },
    collection_name="langchain_example",
    drop_old=False,  # Drop the old Milvus collection if it exists
)

2025-08-26 16:58:57,456 [DEBUG][_create_connection]: Created new connection using: async-http://localhost:19530 (async_milvus_client.py:599)


## 检索

In [4]:
query = "What is self-reflection of an AI Agent?"
vectorstore.similarity_search(query, k=1)

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'pk': 460375432065191400}, page_content='Examples of reasoning trajectories for knowledge-intensive tasks (e.g. HotpotQA, FEVER) and decision-making tasks (e.g. AlfWorld Env, WebShop). (Image source: Yao et al. 2023).\n\nIn both experiments on knowledge-intensive tasks and decision-making tasks, ReAct works better than the Act-only baseline where Thought: … step is removed.\nReflexion (Shinn & Labash 2023) is a framework to equip agents with dynamic memory and self-reflection capabilities to improve reasoning skills. Reflexion has a standard RL setup, in which the reward model provides a simple binary reward and the action space follows the setup in ReAct where the task-specific action space is augmented with language to enable complex reasoning steps. After each action $a_t$, the agent computes a heuristic $h_t$ and optionally may decide to reset the environment to start a new trial depending on the 